- langchain, mongodb, openai(chatGPT), streamlit

    - Mongodb > os에 맞게 설치 필요 > 커뮤니티에디션 설치(무료)

        > 실시간 파이프라인 구성시 데이터 보내고 조회

    - ChatGPT 확인 

    - streamlit.io 

    ![alt text](image.png)

    - Gemini, Claude 사용 가능

- 아래 2가지 모듈을 Prompt Engineering으로 구현

    1. 개체명 인식 & 엔티티 추출 (Named Entity Recognition & Entity Linking)

        - LLM 등장 이후 prompt engineering으로 한 번에 해결 가능 
        - 기존 ) 문장 분류 > 긍/부
        - 변화 ) 문장 > 여러 Entity > Prompt Engineering으로 감성 개별 파악 

        
        > Query + Prompt : `Prompt : 주어진 뉴스 텍스트에서 기업명을 추출하고 ...`

    </br>

    2. Entity 중심의 감성 분석 (Aspect-based Sentiment Analysis)

        - Prompt Engineering으로 구현

        Text --> (prompt) --> Entity Linking & NER 

        Entity 중심의 sentiment 같이 추출하도록 입력 

## NER
- NER : 특정한 개체명(사람 이름, 장소, 조직, 날짜 등)을 식별하고 분류하는 작업 </br></br>
- 목적 </br></br>
    1. 개체명 식별 (Entity Detection) : 텍스트에서 개체명 포함된 부분 식별 </br></br>
    2. 개체명 분류 (Entity Classification) : 식별된 개체명을 사전 정의 카테고리 (사람, 장소, 조직, 날짜 등) 분류

### Spacy

In [ ]:
# spacy (LLM 이전 NLP framework) -- displaCy / BERT > 오류가 있다 (alphabet/meta)
# 분류 모델 / 생성 모델 차이 

In [ ]:
# pip install spacy
# python -m spacy download en_core_web_sm
# python -m spacy download ko_core_news_sm

In [ ]:
import spacy

nlp = spacy.load('en_core_web_sm')

document = 'To study this, we introduce two benchmarks with human ratings as the primary evaluation metric: MT-bench and Chatbot Arena. MT-bench is a series of open-ended questions that evaluate a chatbot’s multi-turn conversational and instruction-following ability – two critical elements for human preference.'

doc = nlp(document)

# Entity 타입 없는 것까지 추출
"""
for e in doc:
    #if e.ent_type_ != "":
    print(f"{e} | {e.ent_type_}")
"""

# Entity 있는 것만 추출
for e in doc:
    if e.ent_type_ != "":
        print(f"{e} | {e.ent_type_}")

In [ ]:
nlp_kr = spacy.load('en_core_web_sm')

document_kr = '네이버가 대형언어모델(LLM) 하이퍼클로바X에 시각 정보 처리 능력을 업데이트,  대형멀티모달모델(LMM)로 업그레이드했다. 지난해 8월24일 하이퍼클로바X 공개 이후 1년만의 메이저 업데이트로, 이제는 외산 인공지능(AI) 챗봇들과 본격적인 경쟁을 펼치겠다는 의도다.'

doc_kr = nlp(document_kr)

# Entity 타입 없는 것까지 추출
"""
for e in doc:
    #if e.ent_type_ != "":
    print(f"{e} | {e.ent_type_}")
"""

# Entity 있는 것만 추출
for e in doc_kr:
    if e.ent_type_ != "":
        print(f"{e} | {e.ent_type_}")

In [ ]:
# Entity Linking (Facebook == Meta)
# 생성 모델은 한 번에 처리할 수 있게 되는 것 

### 생성 NER

In [ ]:
# chatGPT에서 NER 수행

prompt = '''
아래 문장에서 개체명 인식을 수행하세요. 
아래 문장에서 사람 이름 (PERSON), 기업 이름 (ORG)에 해당하는 엔티티만을 추출하세요. 
엔티티의 문장 내 몇 번째 문자 [start, end]를 함께 출력하세요.
출력 결과를 json 포맷으로 {0: {"text":alias, "entity": 분류결과(PERSON/ORG),"position":[0,2]},{1: ...}} 형식으로 출력하세요.

네이버가 대형언어모델(LLM) 하이퍼클로바X에 시각 정보 처리 능력을 업데이트,  대형멀티모달모델(LMM)로 업그레이드했다. 지난해 8월24일 하이퍼클로바X 공개 이후 1년만의 메이저 업데이트로, 이제는 외산 인공지능(AI) 챗봇들과 본격적인 경쟁을 펼치겠다는 의도다.
'''

## Entity Linking (EL)

- 알고자 하는 지식과 연결해주는 작업

    - 기존 DL에서는 많이 사용

    - LLM 등장 이후 지식베이스 힌트 없이도 ChatGPT 이후 가능 

- Entity Linking : 텍스트 내 엔티티를 식별하고, 이를 구조화된 지식베이스의 엔티티와 연결하는 작업

- Entity Ranking 필요성

    1. 정보 검색 측면 : 사용자가 질문한 정보에 대한 정확한 답변 제공

    2. 다양한 엔티티 표현의 통일화 : Facebook, Meta, Meta Platforms, Meta Platforms, Inc.
    
    3. 엔티티 필터링 : 다루고자 하는 지식 베이스에 연결된 엔티티를 필터링해 작업

### Wikification

- Wikification : 텍스트 내 엔티티를 식별하고, 식별된 엔티티를 Wikipedia 항목(지식)과 연결하는 기술

- 지식 베이스르 매번 만들어줘야 하나? `통일화된 합의된 지식`을 사용한다. 

### WikiData
- Wikidata : 위키미디어 재단에서 운영하는, 협력적으로 편집 가능한 지식 베이스 (고유 식별자로 구분)

In [ ]:
# ChatGPT

prompt = '''
주어진 텍스트에 대해서, 지식베이스에 속한 기업을 추출하세요.
지식베이스는 기업명 : 종목코드 형식으로 구성되어 있습니다.
다음과 같은 포맷으로 출력하세요.
{기업명: 종목코드, ...}
반드시 텍스트에 표기되고, 지식베이스에 있는 기업명만을 추출합니다.
지식베이스에 없는 기업은 생성하지 않습니다. 반드시 체크하세요.

#지식베이스
SK하이닉스: 001
삼성바이오로직스 : 002
현대농기계 : 003

# 텍스트
네이버가 대형언어모델(LLM) 하이퍼클로바X에 시각 정보 처리 능력을 업데이트,  대형멀티모달모델(LMM)로 업그레이드했다. 지난해 8월24일 하이퍼클로바X 공개 이후 1년만의 메이저 업데이트로, 이제는 외산 인공지능(AI) 챗봇들과 본격적인 경쟁을 펼치겠다는 의도다.
'''

길이가 길어지면서 할루시네이션 발생.
NER 하고 Entity Linking 하는 방향 

### 분류 기반 감성 분석

In [ ]:
#!pip install transformers
#!pip3 install torch torchvision torchaudio
#conda install pytorch torchvision torchaudio -c pytorch
#conda update -n base -c defaults conda

In [4]:
# HuggingFace > FinBERT > Use this Model
from transformers import pipeline

pipe = pipeline("text-classification", model="ProsusAI/finbert")

In [6]:
print(pipe("네이버가 대형언어모델(LLM) 하이퍼클로바X에 시각 정보 처리 능력을 업데이트,  대형멀티모달모델(LMM)로 업그레이드했다. 지난해 8월24일 하이퍼클로바X 공개 이후 1년만의 메이저 업데이트로, 이제는 외산 인공지능(AI) 챗봇들과 본격적인 경쟁을 펼치겠다는 의도다."))

[{'label': 'neutral', 'score': 0.9039522409439087}]


In [11]:
print(pipe("Meta reported strong fincancial resutls for Q1 2024"))

[{'label': 'positive', 'score': 0.9494152665138245}]


In [13]:
print(pipe("meta reported strong financial results for Q1 2024, but Tesla released its financial result"))

[{'label': 'negative', 'score': 0.8703081011772156}]


- 고전적 방법의 경우 한 문장에 여러 감정이 있는 경우 오류가 날 수 있다. 
- 분류 기반은 정의된 감성에 대해서만 분류되는 한계
- 생성은 강점이 더 많음. 분류 기반은 긍,부,중의 한계. 생성은 고도화된 감정 가능.